# 🔧 Debug Waveform Analyzer Syntax Errors

This notebook helps debug syntax errors in the waveform analyzer script by capturing both stdout and stderr output.

## Goal:
- Execute the waveform analyzer test script
- Capture all output and error messages  
- Identify syntax errors or import issues
- Test the WaveformAnalyzer module functionality

In [ ]:
# Import Required Libraries
import subprocess
import sys
import os
from pathlib import Path

# Set up working directory
workdir = Path.cwd()
print(f"📁 Working directory: {workdir}")
print(f"📝 Python version: {sys.version}")
print(f"🔍 Python executable: {sys.executable}")

In [ ]:
# Test Direct Import
print("🔬 Testing WaveformAnalyzer Import...")
print("=" * 40)

try:
    # Add src to path
    sys.path.insert(0, str(workdir / 'src'))
    print(f"✅ Added to Python path: {workdir / 'src'}")
    
    # Test import
    from waveform_analyzer import WaveformAnalyzer
    print("✅ WaveformAnalyzer imported successfully!")
    
    # Test instance creation
    analyzer = WaveformAnalyzer()
    print("✅ WaveformAnalyzer instance created!")
    
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print(f"📋 Python path: {sys.path}")
except SyntaxError as e:
    print(f"❌ Syntax Error: {e}")
    print(f"📍 File: {e.filename}")
    print(f"📍 Line: {e.lineno}")
except Exception as e:
    print(f"❌ General Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Execute Test Script with Output Capture
print("🚀 Executing Waveform Test Script...")
print("=" * 50)

# Check if test files exist
test_scripts = [
    'test_precision_waveform.py',
    'simple_waveform_test.py'
]

for script in test_scripts:
    script_path = workdir / script
    if script_path.exists():
        print(f"📄 Found test script: {script}")
        
        try:
            # Execute with subprocess and capture output
            result = subprocess.run(
                [sys.executable, str(script_path)],
                cwd=str(workdir),
                capture_output=True,
                text=True,
                timeout=30  # 30 second timeout
            )
            
            print(f"\n🔄 Execution Results for {script}:")
            print(f"   Return code: {result.returncode}")
            
            if result.stdout:
                print(f"\n📤 STDOUT:")
                print(result.stdout)
            else:
                print(f"\n📤 STDOUT: (empty)")
            
            if result.stderr:
                print(f"\n❌ STDERR:")
                print(result.stderr)
            else:
                print(f"\n✅ STDERR: (no errors)")
                
        except subprocess.TimeoutExpired:
            print(f"⏰ Script {script} timed out after 30 seconds")
        except Exception as e:
            print(f"❌ Failed to execute {script}: {e}")
    else:
        print(f"❌ Script not found: {script}")

In [ ]:
# Analyze Module Files for Syntax Errors
print("🔍 Checking Module Files for Syntax Errors...")
print("=" * 50)

import py_compile
import tempfile

# Check key module files
module_files = [
    'src/waveform_analyzer.py',
    'src/enhanced_transcriber.py',
    'test_precision_waveform.py',
    'simple_waveform_test.py'
]

for module_file in module_files:
    file_path = workdir / module_file
    
    if file_path.exists():
        print(f"\n📄 Checking: {module_file}")
        
        try:
            # Use py_compile to check syntax
            with tempfile.NamedTemporaryFile(suffix='.pyc', delete=True) as tmp:
                py_compile.compile(str(file_path), tmp.name, doraise=True)
            print(f"   ✅ Syntax OK")
            
        except py_compile.PyCompileError as e:
            print(f"   ❌ Syntax Error: {e}")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print(f"\n❌ File not found: {module_file}")

In [ ]:
# Test WaveformAnalyzer Functionality
print("🧪 Testing WaveformAnalyzer Functionality...")
print("=" * 50)

try:
    # Ensure import works
    if 'waveform_analyzer' not in sys.modules:
        from waveform_analyzer import WaveformAnalyzer
    
    # Create analyzer instance
    analyzer = WaveformAnalyzer()
    print("✅ WaveformAnalyzer instance created")
    
    # Check for audio file
    audio_files = ['interview_cut.mp3', 'interview.mp3']
    test_audio = None
    
    for audio_file in audio_files:
        if (workdir / audio_file).exists():
            test_audio = str(workdir / audio_file)
            print(f"📁 Found test audio: {audio_file}")
            break
    
    if test_audio:
        print(f"\n🔊 Testing waveform analysis on: {test_audio}")
        
        # Run analysis
        result = analyzer.analyze_speech_segments(test_audio)
        
        print(f"✅ Analysis completed!")
        print(f"📊 Found {len(result)} speech segments")
        
        if result:
            first_seg = result[0]
            last_seg = result[-1]
            print(f"   First segment: {first_seg[0]/1000:.1f}s - {first_seg[1]/1000:.1f}s")
            print(f"   Last segment: {last_seg[0]/1000:.1f}s - {last_seg[1]/1000:.1f}s")
            
            total_speech = sum(end - start for start, end in result) / 1000
            print(f"   Total speech duration: {total_speech:.1f} seconds")
    else:
        print("⚠️ No test audio files found (interview_cut.mp3, interview.mp3)")
        print("   Skipping audio analysis test")
        
except Exception as e:
    print(f"❌ WaveformAnalyzer test failed: {e}")
    import traceback
    traceback.print_exc()

## 📋 Summary

This notebook tested:

1. ✅ **Direct Import Test** - Check if WaveformAnalyzer can be imported
2. ✅ **Script Execution** - Run test scripts and capture output
3. ✅ **Syntax Validation** - Use py_compile to check for syntax errors
4. ✅ **Functionality Test** - Test actual waveform analysis if possible

### Next Steps:
- If syntax errors are found, fix them in the source files
- If import errors occur, check dependencies and module structure
- If functionality tests fail, debug the analysis logic
- Test with actual audio files to validate segmentation results